## Prepare work env

### How to install bigdatasetup (spark, hadoop, jdk)
https://www.youtube.com/watch?v=LrpxfCxmMeo

jdk 11.0.14 (https://www.oracle.com/java/technologies/javase/jdk11-archive-downloads.html)

spark 3.2.3 (https://www.apache.org/dyn/closer.lua/spark/spark-3.2.3/spark-3.2.3-bin-hadoop2.7.tgz)

hadoop 2.7.1 - winutils.exe (https://github.com/steveloughran/winutils/tree/master/hadoop-2.7.1/bin)

### PySpark tutorial
https://www.youtube.com/watch?v=_C8kWso4ne4&t=2268s

### Install requirements.txt
pip install -r requirements.txt

### Setup Env Variables (Windows)
* HADOOP_HOME = <C:\work_space\bigdataseetup\hadoop>
* JAVA_HOME = <C:\Program Files\Java\jdk-11.0.14>
* SPARK_HOME = <C:\work_space\bigdataseetup\spark>

#### Add to PATH env variable
* %JAVA_HOME%\bin
* %HADOOP_HOME%\bin
* %SPARK_HOME%\bin

In [6]:
import pyspark
import pandas as pd

## Init PySpark session

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('projekt2').getOrCreate()

## Read data from csv file

In [8]:
cat_images_dir = "./test_animals/PetImages/Cat/"
dog_images_dir = "./test_animals/PetImages/Dog/"

cat_image_df = spark.read.format("image").load(cat_images_dir).filter("image.nChannels > 2 AND image.height < 1000")
dog_image_df = spark.read.format("image").load(dog_images_dir).filter("image.nChannels > 2 AND image.height < 1000")

cat_image_df.select("image.origin", "image.height", "image.width", "image.mode", "image.nChannels").show(5, truncate=False)

+-----------------------------------------------------------------------------------+------+-----+----+---------+
|origin                                                                             |height|width|mode|nChannels|
+-----------------------------------------------------------------------------------+------+-----+----+---------+
|file:///c:/work_space/studies/MADZD/spark-project/test_animals/PetImages/Cat/23.jpg|257   |334  |16  |3        |
|file:///c:/work_space/studies/MADZD/spark-project/test_animals/PetImages/Cat/46.jpg|434   |400  |16  |3        |
|file:///c:/work_space/studies/MADZD/spark-project/test_animals/PetImages/Cat/59.jpg|331   |465  |16  |3        |
|file:///c:/work_space/studies/MADZD/spark-project/test_animals/PetImages/Cat/27.jpg|480   |370  |16  |3        |
|file:///c:/work_space/studies/MADZD/spark-project/test_animals/PetImages/Cat/47.jpg|214   |259  |16  |3        |
+-----------------------------------------------------------------------------------+---

In [9]:
spark_single_img = cat_image_df.select("image").collect()[1]
print(len( cat_image_df.select("image").collect()))
(spark_single_img.image.origin, spark_single_img.image.mode, spark_single_img.image.nChannels )

66


('file:///c:/work_space/studies/MADZD/spark-project/test_animals/PetImages/Cat/46.jpg',
 16,
 3)

In [10]:
from PIL import Image, ImageDraw

mode = 'RGBA' if (spark_single_img.image.nChannels == 4) else 'RGB' 
Image.frombytes(mode=mode, data=bytes(spark_single_img.image.data), size=[spark_single_img.image.width,spark_single_img.image.height]).show()

In [11]:
import numpy as np

def convert_bgr_array_to_rgb_array(img_array):
    B, G, R = img_array.T
    return np.array((R, G, B)).T

img = Image.frombytes(mode=mode, data=bytes(spark_single_img.image.data), size=[spark_single_img.image.width,spark_single_img.image.height])

converted_img_array = convert_bgr_array_to_rgb_array(np.asarray(img))
Image.fromarray(converted_img_array).show()

NameError: name 'np' is not defined